# Generative Adversarial Networks for Beginners

## Build an artificial neural network that learns to generate handwritten digits.

### By [Jon Bruner](https://github.com/jonbruner) and [Adit Deshpande](https://github.com/adeshpande3)

## Getting Started

We are going to create a GAN that will generate handwritten digits that can fool even the best classifiers (and humans as well).  
We'll use [TensorFlow](https://www.tensorflow.org/), a deep learing library open-sourced by Google that makes it easy to train neural networks on GPUs.

## Loading MNIST Data

We'll need a set of real handwritten digits to give the discriminator a starting point in distinguishing between real and fake images.  
In this tutorial, we are going to use [MNIST](http://yann.lecun.com/exdb/mnist/), a benchmark dataset in deep learning.  
Let us begin by importing TensorFlow along with a couple of other helpful libraries.  
We'll also import our MNIST images using a TensorFlow convenience function called `read_data_sets`.

In [ ]:
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline 

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")